# 01 - BaT Scraping

Scrape completed Porsche 911 auctions from Bring a Trailer.

**Outputs:**
- `data/raw/bat_listings.parquet` - Raw scraped data

In [16]:
# delete bad data if needed
# !rm data/raw/bat_listings.parquet

In [17]:
import logging
import pandas as pd
from pathlib import Path

from price_analysis.scraping import fetch_auctions, validate_scraped_data, DataQualityError
from price_analysis.scraping.bat import listings_to_dataframe

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [18]:
# Paths
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_PATH = RAW_DIR / "bat_listings.parquet"

## Configure Scraping

Adjust search queries and pagination as needed.

In [19]:
# Search queries - multiple generation-specific searches to get more results
# BaT seems to limit how many results a single search returns
QUERIES = [
    # "Porsche 996",  # 1999-2004 (996.1 and 996.2)
    # "Porsche 997",  # 2005-2012 (997.1 and 997.2)
    "Porsche 991",  # 2012-2019 (991.1 and 991.2)
    "Porsche 992",  # 2020+ (992.1 and 992.2)
    "Porsche 911",  # Catch air-cooled + anything else missed
]

# Scraping parameters
MAX_CLICKS = 5  # Number of "Show More" button clicks (each loads ~24 listings)
DELAY = 1.0  # Seconds between requests (be polite!)
HEADLESS = True  # Set False to see browser for debugging

## Debug: Inspect BaT Page Structure (Optional)

Skip this section - fixture files already saved to `tests/fixtures/`.
Only re-run if BaT changes their DOM structure.

In [ ]:
# Debug: Save sample pages for selector inspection
import time
from price_analysis.scraping import create_driver, save_debug_page

# Create driver (set HEADLESS=False above to watch)
driver = create_driver(headless=HEADLESS)

try:
    # 1. Fetch search results page
    search_url = "https://bringatrailer.com/auctions/results/?s=Porsche+911"
    logger.info(f"Fetching search page: {search_url}")
    driver.get(search_url)
    time.sleep(3)  # Wait for JS to load
    save_debug_page(driver, "bat_search_page", output_dir="../tests/fixtures")

    # 2. Fetch a specific listing page (992.1)
    listing_url = "https://bringatrailer.com/listing/2020-porsche-911-carrera-4s-coupe-26/"
    logger.info(f"Fetching listing: {listing_url}")
    driver.get(listing_url)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_992", output_dir="../tests/fixtures")

    # 3. Fetch another listing (997.2) for variety
    listing_url_997 = "https://bringatrailer.com/listing/2009-porsche-911-carrera-4s-coupe-28/"
    logger.info(f"Fetching listing: {listing_url_997}")
    driver.get(listing_url_997)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_997", output_dir="../tests/fixtures")

    logger.info("Debug pages saved to tests/fixtures/")

finally:
    driver.quit()

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/flatljan/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver] found in cache
INFO:__main__:Fetching search page: https://bringatrailer.com/auctions/results/?s=Porsche+911


## Run Scraper

This will take a while depending on MAX_PAGES. Each page + listing takes ~3-5 seconds.

In [ ]:
# Load existing URLs for incremental scraping (skip already-fetched listings)
existing_urls: set[str] = set()
if OUTPUT_PATH.exists():
    existing_df = pd.read_parquet(OUTPUT_PATH)
    existing_urls = set(existing_df["listing_url"])
    logger.info(f"Loaded {len(existing_urls)} existing URLs - will skip these")

all_listings = []

for query in QUERIES:
    logger.info(f"Scraping: {query}")
    listings = fetch_auctions(
        query=query,
        max_clicks=MAX_CLICKS,
        delay=DELAY,
        headless=HEADLESS,
        existing_urls=existing_urls,
    )
    all_listings.extend(listings)
    logger.info(f"Found {len(listings)} NEW listings for '{query}'")

logger.info(f"Total NEW listings scraped: {len(all_listings)}")

## Convert to DataFrame and Save

In [ ]:
df = listings_to_dataframe(all_listings)

# Run data quality checks and filter low-price listings (<$10k)
# Returns filtered DataFrame, raises DataQualityError if checks fail
df = validate_scraped_data(df)

display(df.head(10))
print(f"\nShape: {df.shape}")

INFO:price_analysis.scraping.bat:Filtered 68 listings below $10,000 (likely parts/salvage)
INFO:price_analysis.scraping.bat:Data quality checks passed: 801 listings, 801 with prices


,listing_url,title_raw,sale_price,sale_date,model_year,generation,trim,transmission,mileage,color,location
0,https://bringatrailer.com/listing/2002-porsche...,2002 Porsche 911 Turbo Coupe 6-Speed,69000,2025-12-24,2002.0,996.2,Turbo,Manual,73000.0,Guards Red,"Frisco, Texas 75036"
1,https://bringatrailer.com/listing/2000-porsche...,26k-Mile 2000 Porsche 911 Carrera Cabriolet 6-...,24000,2025-05-13,2000.0,996.1,Carrera,Manual,26000.0,Guards Red,"Waxhaw, North Carolina 28173"
2,https://bringatrailer.com/listing/1999-porsche...,1999 Porsche 911 Carrera Coupe 6-Speed,47246,2025-09-17,1999.0,996.1,Carrera,Manual,59000.0,Black,"Burbank, California 91504"
3,https://bringatrailer.com/listing/1999-porsche...,1999 Porsche 911 Carrera Cabriolet 6-Speed,16250,2025-12-09,1999.0,996.1,Carrera,Manual,110000.0,Arctic Silver Metallic,"Ottawa, Ontario, Canada"
4,https://bringatrailer.com/listing/1999-porsche...,1999 Porsche 911 Carrera Coupe 6-Speed,28650,2025-12-24,1999.0,996.1,Carrera,Manual,71000.0,Vesuvio Metallic,"San Luis Obispo, California 93401"
5,https://bringatrailer.com/listing/2002-porsche...,2002 Porsche 911 Carrera Cabriolet 6-Speed,21000,2025-12-22,2002.0,996.2,Carrera,Manual,99000.0,Lapis Blue Metallic,"Boring, Oregon 97009"
6,https://bringatrailer.com/listing/2003-porsche...,2003 Porsche 911 Carrera Cabriolet 6-Speed,23000,2025-07-24,2003.0,996.2,Carrera,Manual,104000.0,Seal Gray Metallic,"Savannah, Georgia 31405"
7,https://bringatrailer.com/listing/2000-porsche...,36k-Mile 2000 Porsche 911 Carrera Cabriolet 6-...,36666,2025-08-28,2000.0,996.1,Carrera,Manual,36000.0,Mirage Metallic,"Douglassville, Pennsylvania 19518"
8,https://bringatrailer.com/listing/2000-porsche...,2000 Porsche 911 Carrera Cabriolet 6-Speed,20138,2025-11-19,2000.0,996.1,Carrera,Manual,110000.0,Arctic Silver Metallic,"Fort Lauderdale, Florida 33311"
10,https://bringatrailer.com/listing/2004-porsche...,38k-Mile 2004 Porsche 911 Carrera Cabriolet 6-...,24500,2025-05-06,2004.0,996.2,Carrera,Manual,38000.0,Midnight Blue Metallic,"Watkinsville, Georgia 30677"



Shape: (801, 11)


In [ ]:
# Append to existing if present, otherwise create new
if OUTPUT_PATH.exists():
    existing = pd.read_parquet(OUTPUT_PATH)
    df = pd.concat([existing, df], ignore_index=True)
    df = df.drop_duplicates(subset=["listing_url"], keep="last")
    logger.info(f"Merged with existing data: {len(df)} total listings")

df.to_parquet(OUTPUT_PATH, index=False)
logger.info(f"Saved to {OUTPUT_PATH}")

INFO:__main__:Merged with existing data: 818 total listings
INFO:__main__:Saved to ../data/raw/bat_listings.parquet


## Quick Inspection

In [ ]:
print("Counts by generation:")
display(df["generation"].value_counts(dropna=False))

Counts by generation:


generation
997.1    273
996.1    186
996.2    169
997.2    144
None      36
991.1      5
991.2      2
992.1      2
992.2      1
Name: count, dtype: int64

In [ ]:
print("Counts by trim:")
display(df["trim"].value_counts(dropna=False))

Counts by trim:


trim
Carrera        235
Turbo          150
Carrera S      118
Carrera 4S     117
Carrera 4       59
None            42
GT3             29
Turbo S         29
GT3 RS          13
Targa 4S        11
Targa            9
Targa 4          4
GT3 Touring      1
GT2 RS           1
Name: count, dtype: int64

In [ ]:
print("Counts by transmission:")
display(df["transmission"].value_counts(dropna=False))

Counts by transmission:


transmission
Manual       626
PDK           72
Tiptronic     50
Automatic     41
None          29
Name: count, dtype: int64

In [ ]:
# Check parsing quality - how many have all required fields?
required = ["sale_price", "model_year", "generation", "trim", "transmission", "mileage"]
complete = df[required].notna().all(axis=1).sum()
print(
    f"\nListings with all required fields: {complete} / {len(df)} ({complete / len(df) * 100:.1f}%)"
)


Listings with all required fields: 756 / 818 (92.4%)


In [ ]:
# Sample some listings to verify parsing
print("Sample listings for manual verification:")
sample = df.sample(min(10, len(df)), random_state=42)
for _, row in sample.iterrows():
    print(f"\n{row['title_raw']}")
    print(
        f"  Parsed: {row['model_year']} {row['generation']} {row['trim']} ({row['transmission']})"
    )
    print(f"  Price: ${row['sale_price']:,}" if pd.notna(row["sale_price"]) else "  Price: N/A")
    print(f"  Mileage: {row['mileage']:,}" if pd.notna(row["mileage"]) else "  Mileage: N/A")

Sample listings for manual verification:

43k-Mile 2000 Porsche 911 Carrera Cabriolet 6-Speed
  Parsed: 2000.0 996.1 Carrera (Manual)
  Price: $22,000
  Mileage: 43,000.0

2012 Porsche 911 Carrera S Coupe 6-Speed
  Parsed: 2012.0 997.2 Carrera S (Manual)
  Price: $66,744
  Mileage: 73,000.0

2007 Porsche 911 GT3 Cup
  Parsed: 2007.0 997.1 GT3 (None)
  Price: $102,997
  Mileage: 9,000.0

2005 Porsche 911 Carrera Coupe 6-Speed
  Parsed: 2005.0 997.1 Carrera (Manual)
  Price: $32,000
  Mileage: 91,000.0

35k-Mile 2006 Porsche 911 Carrera S Cabriolet 6-Speed
  Parsed: 2006.0 997.1 Carrera S (Manual)
  Price: $59,055
  Mileage: 35,000.0

19k-Mile 2007 Porsche 911 Turbo Coupe 6-Speed
  Parsed: 2007.0 997.1 Turbo (Manual)
  Price: $129,333
  Mileage: 19,000.0

2013 Porsche 911 Turbo S Coupe
  Parsed: 2013.0 991.1 Turbo S (PDK)
  Price: $84,000
  Mileage: 73,000.0

2006 Porsche 911 Carrera S Coupe 6-Speed
  Parsed: 2006.0 997.1 Carrera S (Manual)
  Price: $57,911
  Mileage: 67,000.0

2001 Pors